In [25]:
from dotenv import load_dotenv
import os
from main import (
    add_metric,
    remove_metric,
    get_metrics,
    validate_user_input,
    export_to_excel,
    create_human_review_template,
)
import pandas as pd

In [26]:
env_path = "/Users/ford/Documents/coding/confidential/.env"
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")
assert api_key, "API key is missing"

In [27]:
risk_catalog_df_path = (
    "/Users/ford/Documents/coding/confidential/RMI-V2-Translate_20250213_ford_edit.xlsx"
)
assert os.path.exists(risk_catalog_df_path)

In [28]:
risk_catalog_df = pd.read_excel(risk_catalog_df_path)
# where Risk-category is 'Operational risk' and Operation is 'Operation'
risk_catalog_df = risk_catalog_df[
    (risk_catalog_df["Risk-category"] == "Operational risk")
    & (risk_catalog_df["Operation"] == "Operation")
]
risk_catalog_df.info()

/Users/ford/Documents/coding_trae/cro_rmi_improvement_feature/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


<class 'pandas.core.frame.DataFrame'>
Index: 65 entries, 0 to 64
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   GroupingIndex           65 non-null     int64 
 1   Grouping Question Name  65 non-null     object
 2   Code                    65 non-null     object
 3   Risk-category           65 non-null     object
 4   Operation               65 non-null     object
 5   Risk-EN                 65 non-null     object
 6   Risk-TH                 65 non-null     object
 7   Risk-CN                 65 non-null     object
 8   Description-EN          65 non-null     object
 9   Description-TH          65 non-null     object
 10  Description-CN          65 non-null     object
 11  Unnamed: 11             0 non-null      object
 12  Unnamed: 12             0 non-null      object
dtypes: int64(1), object(12)
memory usage: 7.1+ KB


In [29]:
risk_catalog_df["Description-EN"].nunique()

65

In [30]:
risk_catalog_df["Description-EN"].shape

(65,)

In [31]:
data_path = "/Users/ford/Documents/coding/confidential/250327_data_from_RMI_from_productionbuild_Q3-2024_label.xlsx"

# assert is file exist
assert os.path.exists(data_path)

In [32]:
data_df = pd.read_excel(data_path)
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1783 entries, 0 to 1782
Data columns (total 82 columns):
 #   Column                                                        Non-Null Count  Dtype  
---  ------                                                        --------------  -----  
 0   id                                                            789 non-null    float64
 1   control_name                                                  789 non-null    object 
 2   control_owner                                                 789 non-null    object 
 3   have_control                                                  1783 non-null   bool   
 4   control_freq                                                  0 non-null      float64
 5   control_effective_date                                        789 non-null    object 
 6   risk_question_id                                              1783 non-null   int64  
 7   control_des                                                   789 non

In [33]:
data_df.shape

(1783, 82)

In [34]:
# selected_df = data_df[data_df["iscustom"] == True]
selected_df = data_df.copy()
# column risk_desc unique values

risk_desc_unique_list = selected_df["risk_desc"].unique().tolist()
risk_name_dict = {}
# if not nann
risk_desc_unique_list = [x for x in risk_desc_unique_list if not pd.isna(x)]
# Iterate through the unique risk descriptions
for risk_desc in risk_desc_unique_list:
    # Find the corresponding risk name. We take the first one if there are duplicates
    try:
        risk_name = selected_df[selected_df["risk_desc"] == risk_desc]["risk"].iloc[0]
    except IndexError:
        print(f"Warning: No risk name found for risk description '{risk_desc}'")
        raise
    risk_name_dict[risk_desc] = risk_name

# print(risk_name_dict)
assert len(risk_desc_unique_list) == len(
    risk_name_dict
), f"risk_desc_unique_list and risk_name_dict should have the same length but got {len(risk_desc_unique_list)} and {len(risk_name_dict)}"
control_desc_unique_list = selected_df["control_des"].unique().tolist()
selected_df.shape

(1783, 82)

In [35]:
selected_control_df = selected_df[["control_name", "control_des", "risk", "risk_desc"]]
# remove row that control_name is duplicated
selected_control_df = selected_control_df.drop_duplicates(
    subset=["control_name"], keep="first"
)
selected_control_df.head()

,control_name,control_des,risk,risk_desc
0,NaN,NaN,Debtor credit risk (Internal party),ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน
57,กำหนดมาตรการในการเบิกค่าใช้จ่าย,1. พนักงานเบิกค่าใช้จ่ายต้องจัดทำ Request ให้ผ...,การทุจริตในกระบวนการทำงาน (AcTech),การทุจริตในกระบวน การเบิกจ่ายค่าใช้จ่าย การจัด...
58,การเปรียบเทียบราคาของ Supplier อย่างน้อย 3 ราย,เปรียบเทียบโดยดูจากความพร้อม ราคา ความชำนาญ แล...,การทุจริตในกระบวนการทำงาน (AcTech),การทุจริตในกระบวน การเบิกจ่ายค่าใช้จ่าย การจัด...
59,DOA (Delegation of Authority),อำนาจการอนุมัติ ตามตำแหน่งงาน (Set up เข้าระบบ...,การทุจริตในกระบวนการทำงาน (AcTech),การทุจริตในกระบวน การเบิกจ่ายค่าใช้จ่าย การจัด...
60,ปรับปรุงกระบวนการเบิกของส่งเสริมการขาย ในระบบ ...,1. พนักงานต้องการเบิกสินค้าไปจัดกิจกรรมส่งเสริ...,การทุจริตในกระบวนการทำงาน (Fert),1.การทุจริตในกระบวนการขาย การเบิกจ่าย ของส่งเส...


In [36]:
# len(risk_desc_unique_list), len(control_desc_unique_list), len(risk_name_unique_list)

In [37]:
# risk_desc_unique_list = risk_desc_unique_list[:1]
# risk_desc_unique_list = risk_desc_unique_list[:2]

# risk_desc_unique_list = risk_desc_unique_list[:10]

In [38]:
len(risk_desc_unique_list)

1368

In [39]:
# risk
user_inputs = risk_desc_unique_list
additional_data_list = None

In [40]:
# # control
# n_rows = 10
# input_data_df = selected_control_df.head(n_rows)
# user_inputs = input_data_df["control_des"].to_list()
# risk_names = input_data_df["risk"].to_list()
# risk_descs = input_data_df["risk_desc"].to_list()
# additional_data_list = []
# for i in range(n_rows):
#     additional_data_list.append(
#         {
#             "risk_name": risk_names[i],
#             "risk_desc": risk_descs[i],
#         }
#     )

In [41]:
# # risk catalog
# user_inputs = risk_catalog_df["Description-EN"].to_list()
# risk_name = risk_catalog_df["Risk-EN"].to_list()
# # check is risk_name is unique
# assert len(risk_name) == len(set(risk_name))
# # check is user_inputs is unique
# assert len(user_inputs) == len(set(user_inputs))
# risk_name_dict = {}  # map key user_inputs to risk_name
# for i in range(len(user_inputs)):
#     risk_name_dict[user_inputs[i]] = risk_name[i]

# # user_inputs = user_inputs[:3]
# additional_data_list = None
# len(user_inputs)

In [42]:
# Initialize default metrics risk
# add_metric(
#     "Have cause",
#     "Text identifies and explains the underlying root cause or triggering factors of the risk",
# )
add_metric(
    "Have risk event",
    "Text clearly describes what could go wrong or the specific risk scenario that may occur",
)
add_metric(
    "Have single risk event",
    "Text describes a single, specific risk event rather than multiple scenarios or vague possibilities",
)
add_metric(
    "Have impact",
    "Text describes potential consequences or outcomes resulting from the risk event",
)

{'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
 'Have single risk event': 'Text describes a single, specific risk event rather than multiple scenarios or vague possibilities',
 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}

In [43]:
# # control
# add_metric(
#     "Have clear objective",
#     "Text clearly states the specific goal or purpose of the control",
# )
# add_metric(
#     "Have detailed mechanism",
#     "Text explains the specific activities, steps, or methods used to implement the control",
# )
# add_metric(
#     "Have quantified measures",
#     "Text includes specific, measurable elements or thresholds where applicable",
# )
# add_metric(
#     "Have dependencies",
#     "Text identifies any relationships or dependencies with other processes or controls",
# )

In [44]:
get_metrics()

{'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
 'Have single risk event': 'Text describes a single, specific risk event rather than multiple scenarios or vague possibilities',
 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}

In [45]:
from langchain.callbacks import get_openai_callback

In [46]:
parts = 5
part_ind = 0  # 0, 1, 2, 3, 4
# make user_inputs split into parts
user_inputs_split = user_inputs[
    part_ind * len(user_inputs) // parts : (part_ind + 1) * len(user_inputs) // parts
]
len(user_inputs_split)

273

In [47]:
# 1783 / 274

In [48]:
user_inputs = user_inputs_split

In [49]:
all_result = []
all_example_text = []
all_feedback = []
if additional_data_list == None:
    additional_data_list = [None] * len(user_inputs)
fail_list = []
with get_openai_callback() as cb:

    for user_input, additional_data in zip(user_inputs, additional_data_list):
        try:
            result, improved_example_text, feedback_dict = validate_user_input(
                user_input, additional_information=additional_data
            )
            all_result.append(result)
            all_example_text.append(improved_example_text)
            all_feedback.append(feedback_dict)
        except Exception as e:
            fail_list.append(user_input)

Validating user input: 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน'
Using provider: openai, model: default
Metrics: {'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur', 'Have single risk event': 'Text describes a single, specific risk event rather than multiple scenarios or vague possibilities', 'Have impact': 'Text describes potential consequences or outcomes resulting from the risk event'}
Validating text on metric: Have risk event
Validating text on metric: Have single risk event
Validating text on metric: Have impact
Validating user input: 'มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เปิดอีเมล์จากภายนอกอาจทำให้เกิดไวรัสได้'
Using provider: openai, model: default
Metrics: {'Have risk event': 'Text clearly describes what could go wrong or the specific risk scenario that may occur', 'Have single risk event': 'Text describes a single, specific risk event rather than multiple scenarios or vague possibilities', 'Have impact': 'Text descr

In [50]:
len(user_inputs)

273

In [51]:
cb

Tokens Used: 509424
	Prompt Tokens: 353204
		Prompt Tokens Cached: 0
	Completion Tokens: 156220
		Reasoning Tokens: 0
Successful Requests: 1249
Total Cost (USD): $0.14671260000000008

In [52]:
0.09450689999999998 * 35

3.3077414999999992

In [53]:
all_example_text

[{'improved_example_text': {'invalid_metrics': ['Have impact'],
   'examples': {'choice1': 'การเลื่อนการชำระเงินจากลูกหนี้บางรายอาจส่งผลกระทบต่อกระแสเงินสดของบริษัท ทำให้เกิดความเสี่ยงทางการเงินที่ต้องพิจารณาอย่างรอบคอบ.',
    'choice2': 'หากลูกหนี้บางรายขอเลื่อนการชำระเงิน อาจส่งผลกระทบต่อความสามารถในการดำเนินงานของบริษัท และสร้างความเสี่ยงที่อาจทำให้เกิดปัญหาทางการเงินในอนาคต.',
    'choice3': 'การที่ลูกหนี้บางรายขอเลื่อนการชำระเงินอาจมีผลกระทบต่อการจัดการเงินสดของบริษัท ซึ่งอาจนำไปสู่ความเสี่ยงทางการเงินที่สำคัญที่ต้องได้รับการจัดการ.'},
   'original_text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน'}},
 {'improved_example_text': {'invalid_metrics': ['Have single risk event'],
   'examples': {'choice1': 'มีความเสี่ยงจากการรับหรือเปิดอีเมล์จากภายนอก ซึ่งอาจนำไปสู่การติดไวรัสในระบบได้ หากไม่ระมัดระวังในการตรวจสอบแหล่งที่มาของอีเมล์เหล่านั้น',
    'choice2': 'การรับและเปิดอีเมล์จากภายนอกมีความเสี่ยงที่อาจทำให้เกิดการติดไวรัสในระบบได้ โดยเฉพาะเมื่ออีเมล์มาจากแหล่งที่ไม่น่าเชื่อถือ',
    'choic

In [54]:
all_feedback

[{'summary_paragraph': "The user's sentence regarding debtors requesting to postpone payments lacks depth in discussing the potential consequences of such actions, which is essential for a comprehensive understanding of the associated risks.",
  'reasons': [{'reason_title': 'Insufficient Impact Analysis',
    'reason_description': 'The sentence mentions that some debtors may request to postpone payments but fails to explore the potential consequences of this action. Without discussing impacts such as cash flow issues, increased debt, or legal implications, the statement lacks the necessary depth to convey the seriousness of the situation.',
    'related_metrics': ['Have impact'],
    'improvement_suggestion': 'Enhance the sentence by including specific potential outcomes of postponing payments, such as effects on cash flow, increased financial burden, or possible legal repercussions.'}],
  'original_text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน'},
 {'summary_paragraph': "The user's senten

In [55]:
0.0011229 / 2 * 35 * 282

5.5415115

In [56]:
all_result[:3]

[[{'text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน',
   'metric': 'Have risk event',
   'metric_detail': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
   'additional_information': None,
   'is_valid': True,
   'reason': 'The text clearly describes a specific risk scenario where some debtors may request to postpone their payment, which indicates a potential risk event related to cash flow and financial management.',
   'provider': 'openai',
   'model': 'default'},
  {'text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน',
   'metric': 'Have single risk event',
   'metric_detail': 'Text describes a single, specific risk event rather than multiple scenarios or vague possibilities',
   'additional_information': None,
   'is_valid': True,
   'reason': 'The text describes a specific risk event where certain debtors request to postpone their payment. This is a clear and singular scenario, rather than a vague possibility or multiple scenarios.',
   'provider': 'o

In [57]:
len(all_result)

273

In [58]:
len(all_result)

273

In [59]:
feedback_dict.keys()

dict_keys(['summary_paragraph', 'reasons', 'original_text'])

In [60]:
feedback_dict["summary_paragraph"]

"The user's sentence lacks clarity and specificity regarding potential risk events associated with intercompany loans and accrued interest income. It fails to articulate specific risks and their potential impacts, which are essential for a comprehensive risk assessment."

In [61]:
len(feedback_dict["reasons"])

2

In [62]:
feedback_dict["reasons"][0]

{'reason_title': 'Lack of Clear and Specific Risk Identification',
 'reason_description': 'The sentence does not clearly identify specific risk events related to the financial figures presented. Instead of outlining distinct risks such as default risk or liquidity risk, it provides a general financial overview without pinpointing what could go wrong.',
 'related_metrics': ['Have risk event', 'Have single risk event'],
 'improvement_suggestion': 'Revise the sentence to explicitly state a specific risk event associated with the intercompany loans and accrued interest income, such as the risk of default by the related party.'}

In [63]:
# all_feedback

In [64]:
new_all_feedback = []
for i in all_feedback:
    if i != {}:
        new_all_feedback.append(i)

In [65]:
new_all_example_text = []
for i in all_example_text:
    if i != {}:
        new_all_example_text.append(i)

In [66]:
new_all_example_text

[{'improved_example_text': {'invalid_metrics': ['Have impact'],
   'examples': {'choice1': 'การเลื่อนการชำระเงินจากลูกหนี้บางรายอาจส่งผลกระทบต่อกระแสเงินสดของบริษัท ทำให้เกิดความเสี่ยงทางการเงินที่ต้องพิจารณาอย่างรอบคอบ.',
    'choice2': 'หากลูกหนี้บางรายขอเลื่อนการชำระเงิน อาจส่งผลกระทบต่อความสามารถในการดำเนินงานของบริษัท และสร้างความเสี่ยงที่อาจทำให้เกิดปัญหาทางการเงินในอนาคต.',
    'choice3': 'การที่ลูกหนี้บางรายขอเลื่อนการชำระเงินอาจมีผลกระทบต่อการจัดการเงินสดของบริษัท ซึ่งอาจนำไปสู่ความเสี่ยงทางการเงินที่สำคัญที่ต้องได้รับการจัดการ.'},
   'original_text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน'}},
 {'improved_example_text': {'invalid_metrics': ['Have single risk event'],
   'examples': {'choice1': 'มีความเสี่ยงจากการรับหรือเปิดอีเมล์จากภายนอก ซึ่งอาจนำไปสู่การติดไวรัสในระบบได้ หากไม่ระมัดระวังในการตรวจสอบแหล่งที่มาของอีเมล์เหล่านั้น',
    'choice2': 'การรับและเปิดอีเมล์จากภายนอกมีความเสี่ยงที่อาจทำให้เกิดการติดไวรัสในระบบได้ โดยเฉพาะเมื่ออีเมล์มาจากแหล่งที่ไม่น่าเชื่อถือ',
    'choic

In [67]:
new_all_feedback

[{'summary_paragraph': "The user's sentence regarding debtors requesting to postpone payments lacks depth in discussing the potential consequences of such actions, which is essential for a comprehensive understanding of the associated risks.",
  'reasons': [{'reason_title': 'Insufficient Impact Analysis',
    'reason_description': 'The sentence mentions that some debtors may request to postpone payments but fails to explore the potential consequences of this action. Without discussing impacts such as cash flow issues, increased debt, or legal implications, the statement lacks the necessary depth to convey the seriousness of the situation.',
    'related_metrics': ['Have impact'],
    'improvement_suggestion': 'Enhance the sentence by including specific potential outcomes of postponing payments, such as effects on cash flow, increased financial burden, or possible legal repercussions.'}],
  'original_text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน'},
 {'summary_paragraph': "The user's senten

In [68]:
import pandas as pd


def convert_all_example_text_to_df(data):
    """
    Converts a list of dictionaries with a specific nested structure to an Excel file
    where each 'choice' becomes a separate row.

    Args:
        data (list): A list of dictionaries with the structure described previously.
        output_filename (str, optional): The name of the Excel file to create.
                                         Defaults to 'output_row_format.xlsx'.

    Returns:
        str: A message indicating the successful creation of the Excel file.
    """
    excel_data = []

    for item in data:
        improved_data = item.get("improved_example_text", {})
        original_text = improved_data.get("original_text")
        invalid_metrics = improved_data.get("invalid_metrics", [])
        examples = improved_data.get("examples", {})

        for choice_key, choice_value in examples.items():
            excel_data.append(
                {
                    "original_text": original_text,
                    "choice_key": choice_key,
                    "choice_value": choice_value,
                    "invalid_metrics": (
                        ", ".join(invalid_metrics) if invalid_metrics else None
                    ),
                }
            )

    # Create a Pandas DataFrame
    df = pd.DataFrame(excel_data)

    return df


# Example usage:

all_example_text_df = convert_all_example_text_to_df(all_example_text)

all_example_text_df.shape

(645, 4)

In [69]:
import pandas as pd
import json


def convert_all_feedback_to_df(data):
    """
    Converts a list of dictionaries with a specific structure to a Pandas DataFrame
    with columns 'original text', 'raw_data', and 'data_as_text'.

    Args:
        data (list): A list of dictionaries with the structure described previously.

    Returns:
        pandas.DataFrame: The created Pandas DataFrame.
    """
    excel_data = []

    for item in data:
        original_text = item.get("original_text")
        raw_data = json.dumps(item)
        summary_paragraph = item.get("summary_paragraph", "")
        reasons = item.get("reasons", [])

        data_as_text = f"Summary: {summary_paragraph}\nReasons:"
        if reasons:
            for i, reason in enumerate(reasons):
                reason_description = reason.get("reason_description", "")
                data_as_text += f"\n{i+1}. {reason_description}"
        else:
            data_as_text += "\nNo reasons provided."

        excel_data.append(
            {
                "original text": original_text,
                "raw_data": raw_data,
                "data_as_text": data_as_text,
            }
        )

    # Create a Pandas DataFrame
    df = pd.DataFrame(excel_data)

    return df


# Example usage:

# Convert to DataFrame and print it
all_feedback_df = convert_all_feedback_to_df(new_all_feedback)
print("DataFrame:")
all_feedback_df

DataFrame:


,original text,raw_data,data_as_text
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,"{""summary_paragraph"": ""The user's sentence reg...",Summary: The user's sentence regarding debtors...
1,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
2,อาจจะมีการเปิดไฟล์แนบมาในเมล์ที่มีไวรัสทำให้เก...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
3,เกิดการโจรกรรม ไฟล์และข้อมูลลูกค้าเปิดบริการเกมส์,"{""summary_paragraph"": ""The user's sentence des...",Summary: The user's sentence describes a risk ...
4,ระบบต้องได้รับการอัพเดต รวมไปถึงเทคโนโลยีใหม่ๆ...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not meet the ...
...,...,...,...
210,Concentration ของ source of revenue กับลูกค้าใ...,"{""summary_paragraph"": ""The user's sentence reg...",Summary: The user's sentence regarding the con...
211,ลูกค้า (end customer) ซื้อสินค้าน้อยลง และกลุ่...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
212,COVID 19,"{""summary_paragraph"": ""The user sentence 'COVI...",Summary: The user sentence 'COVID 19' did not ...
213,"Analytics, Media, and CRM services are commonl...","{""summary_paragraph"": ""The user's sentence fai...",Summary: The user's sentence fails to meet the...


In [70]:
all_feedback_df.data_as_text.to_list()[0]

"Summary: The user's sentence regarding debtors requesting to postpone payments lacks depth in discussing the potential consequences of such actions, which is essential for a comprehensive understanding of the associated risks.\nReasons:\n1. The sentence mentions that some debtors may request to postpone payments but fails to explore the potential consequences of this action. Without discussing impacts such as cash flow issues, increased debt, or legal implications, the statement lacks the necessary depth to convey the seriousness of the situation."

In [71]:
new_all_result = []
for i in all_result:
    new_all_result.extend(i)

In [72]:
new_all_result[0]

{'text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน',
 'metric': 'Have risk event',
 'metric_detail': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
 'additional_information': None,
 'is_valid': True,
 'reason': 'The text clearly describes a specific risk scenario where some debtors may request to postpone their payment, which indicates a potential risk event related to cash flow and financial management.',
 'provider': 'openai',
 'model': 'default'}

In [73]:
all_result_df = create_human_review_template(new_all_result)

In [74]:
def create_debug_df(data_list: list[dict]) -> pd.DataFrame:
    data_dict = {}
    all_metrics = []
    for data in data_list:
        text = data["text"]
        data_dict[text] = data_dict.get(text, {})
        metric_name = data["metric"]
        is_valid = data["is_valid"]
        data_dict[text][metric_name] = is_valid
        if metric_name not in all_metrics:
            all_metrics.append(metric_name)
    for data in data_list:
        text = data["text"]
        for metric in all_metrics:
            if metric not in data_dict[text]:
                data_dict[text][metric] = None
    df = pd.DataFrame(data_dict).transpose()
    # more index to original text
    df.index.name = "original_text"
    df.reset_index(inplace=True)

    return df


all_debug_df = create_debug_df(new_all_result)
all_debug_df.head()

,original_text,Have risk event,Have single risk event,Have impact
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,True,True,False
1,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,True,False,True
2,การถูกโจมตีจากกระบวนการของคนนอก เช่น โจมตี server,True,True,True
3,อาจจะมีการเปิดไฟล์แนบมาในเมล์ที่มีไวรัสทำให้เก...,True,False,True
4,ความเสี่ยงต่อการโจมตีอุปกรณ์ไอทีภายในองค์กร ที...,True,True,True


In [75]:
import pandas as pd
import time


def save_dataframes_to_excel_from_dict(dict_of_dataframes, excel_filename=None):
    """
    Saves a dictionary of Pandas DataFrames to different sheets in a single Excel file.
    The keys of the dictionary will be used as the sheet names.

    Args:
        dict_of_dataframes (dict): A dictionary where keys are sheet names (str)
                                    and values are Pandas DataFrames.
        excel_filename (str, optional): The name of the Excel file to create.
                                         Defaults to 'output.xlsx'.

    Returns:
        None
    """
    if excel_filename is None:
        time_stampe = time.strftime("%Y%m%d-%H%M%S")
        excel_filename = f"output_{time_stampe}.xlsx"
    if not isinstance(dict_of_dataframes, dict):
        raise TypeError("Input 'dict_of_dataframes' must be a dictionary.")
    if not all(isinstance(df, pd.DataFrame) for df in dict_of_dataframes.values()):
        raise ValueError(
            "All values in 'dict_of_dataframes' must be Pandas DataFrames."
        )

    with pd.ExcelWriter(excel_filename) as writer:
        for sheet_name, df in dict_of_dataframes.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(
        f"Successfully saved {len(dict_of_dataframes)} DataFrames to '{excel_filename}' with sheets: {', '.join(dict_of_dataframes.keys())}"
    )


dict_of_dataframes = {
    "label_metric": all_result_df,
    "debug_metric": all_debug_df,
    "improved_example_sentences": all_example_text_df,
    "feedback_messages": all_feedback_df,
}

In [76]:
# save_dataframes_to_excel_from_dict(dict_of_dataframes)

In [77]:
"complete"

'complete'

In [78]:
all_result_df

,text,metric,metric_detail,additional_information,is_valid,reason,provider,model,human_is_valid,human_reason
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,Have risk event,Text clearly describes what could go wrong or ...,None,True,The text clearly describes a specific risk sce...,openai,default,None,None
1,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,Have single risk event,"Text describes a single, specific risk event r...",None,True,The text describes a specific risk event where...,openai,default,None,None
2,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,Have impact,Text describes potential consequences or outco...,None,False,The text states that some debtors may request ...,openai,default,None,None
3,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,Have risk event,Text clearly describes what could go wrong or ...,None,True,The text clearly describes a specific risk sce...,openai,default,None,None
4,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,Have single risk event,"Text describes a single, specific risk event r...",None,False,The text describes a general risk related to r...,openai,default,None,None
...,...,...,...,...,...,...,...,...,...,...
814,"Analytics, Media, and CRM services are commonl...",Have single risk event,"Text describes a single, specific risk event r...",None,False,"The text does not describe a single, specific ...",openai,default,None,None
815,"Analytics, Media, and CRM services are commonl...",Have impact,Text describes potential consequences or outco...,None,False,The text does not describe potential consequen...,openai,default,None,None
816,PTVN has interco-loan and accured interest inc...,Have risk event,Text clearly describes what could go wrong or ...,None,False,The text does not clearly describe a specific ...,openai,default,None,None
817,PTVN has interco-loan and accured interest inc...,Have single risk event,"Text describes a single, specific risk event r...",None,False,The text describes a financial situation invol...,openai,default,None,None


In [79]:
new_all_result[:3]

[{'text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน',
  'metric': 'Have risk event',
  'metric_detail': 'Text clearly describes what could go wrong or the specific risk scenario that may occur',
  'additional_information': None,
  'is_valid': True,
  'reason': 'The text clearly describes a specific risk scenario where some debtors may request to postpone their payment, which indicates a potential risk event related to cash flow and financial management.',
  'provider': 'openai',
  'model': 'default'},
 {'text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน',
  'metric': 'Have single risk event',
  'metric_detail': 'Text describes a single, specific risk event rather than multiple scenarios or vague possibilities',
  'additional_information': None,
  'is_valid': True,
  'reason': 'The text describes a specific risk event where certain debtors request to postpone their payment. This is a clear and singular scenario, rather than a vague possibility or multiple scenarios.',
  'provider': 'openai',
  'mode

In [80]:
all_debug_df

,original_text,Have risk event,Have single risk event,Have impact
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,True,True,False
1,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,True,False,True
2,การถูกโจมตีจากกระบวนการของคนนอก เช่น โจมตี server,True,True,True
3,อาจจะมีการเปิดไฟล์แนบมาในเมล์ที่มีไวรัสทำให้เก...,True,False,True
4,ความเสี่ยงต่อการโจมตีอุปกรณ์ไอทีภายในองค์กร ที...,True,True,True
...,...,...,...,...
268,Concentration ของ source of revenue กับลูกค้าใ...,False,False,False
269,ลูกค้า (end customer) ซื้อสินค้าน้อยลง และกลุ่...,True,False,True
270,COVID 19,False,False,False
271,"Analytics, Media, and CRM services are commonl...",False,False,False


In [81]:
all_example_text_df

,original_text,choice_key,choice_value,invalid_metrics
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,choice1,การเลื่อนการชำระเงินจากลูกหนี้บางรายอาจส่งผลกร...,Have impact
1,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,choice2,หากลูกหนี้บางรายขอเลื่อนการชำระเงิน อาจส่งผลกร...,Have impact
2,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,choice3,การที่ลูกหนี้บางรายขอเลื่อนการชำระเงินอาจมีผลก...,Have impact
3,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,choice1,มีความเสี่ยงจากการรับหรือเปิดอีเมล์จากภายนอก ซ...,Have single risk event
4,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,choice2,การรับและเปิดอีเมล์จากภายนอกมีความเสี่ยงที่อาจ...,Have single risk event
...,...,...,...,...
640,"Analytics, Media, and CRM services are commonl...",choice2,"In the competitive landscape of Analytics, Med...","Have risk event, Have single risk event, Have ..."
641,"Analytics, Media, and CRM services are commonl...",choice3,"The delivery of Analytics, Media, and CRM serv...","Have risk event, Have single risk event, Have ..."
642,PTVN has interco-loan and accured interest inc...,choice1,PTVN faces a significant risk event involving ...,"Have risk event, Have single risk event, Have ..."
643,PTVN has interco-loan and accured interest inc...,choice2,The risk associated with PTVN's intercompany l...,"Have risk event, Have single risk event, Have ..."


In [82]:
# convert all_example_text_df to json or list
all_example_text_df.to_dict("records")

[{'original_text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน',
  'choice_key': 'choice1',
  'choice_value': 'การเลื่อนการชำระเงินจากลูกหนี้บางรายอาจส่งผลกระทบต่อกระแสเงินสดของบริษัท ทำให้เกิดความเสี่ยงทางการเงินที่ต้องพิจารณาอย่างรอบคอบ.',
  'invalid_metrics': 'Have impact'},
 {'original_text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน',
  'choice_key': 'choice2',
  'choice_value': 'หากลูกหนี้บางรายขอเลื่อนการชำระเงิน อาจส่งผลกระทบต่อความสามารถในการดำเนินงานของบริษัท และสร้างความเสี่ยงที่อาจทำให้เกิดปัญหาทางการเงินในอนาคต.',
  'invalid_metrics': 'Have impact'},
 {'original_text': 'ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน',
  'choice_key': 'choice3',
  'choice_value': 'การที่ลูกหนี้บางรายขอเลื่อนการชำระเงินอาจมีผลกระทบต่อการจัดการเงินสดของบริษัท ซึ่งอาจนำไปสู่ความเสี่ยงทางการเงินที่สำคัญที่ต้องได้รับการจัดการ.',
  'invalid_metrics': 'Have impact'},
 {'original_text': 'มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เปิดอีเมล์จากภายนอกอาจทำให้เกิดไวรัสได้',
  'choice_key': 'choice1',
  'choice_value': 'มีควา

In [83]:
import json

data = all_example_text_df.to_dict("records")

output_data = {}

for item in data:
    original_text = item["original_text"]
    choice_key = item["choice_key"]
    choice_value = item["choice_value"]
    invalid_metrics = item["invalid_metrics"]

    if original_text not in output_data:
        output_data[original_text] = {}

    choice_number = choice_key.replace("choice", "")
    output_data[original_text][f"choice_{choice_number}-value"] = choice_value
    output_data[original_text][
        f"choice_{choice_number}-invalid_metrics"
    ] = invalid_metrics

# Convert the dictionary to the desired list of dictionaries format
result = []
for original_text, choices in output_data.items():
    output_item = {"original_text": original_text}
    output_item.update(choices)
    result.append(output_item)

# Print the resulting list of dictionaries (optional)
new_all_example_text_df = pd.DataFrame(result)
new_all_example_text_df

,original_text,choice_1-value,choice_1-invalid_metrics,choice_2-value,choice_2-invalid_metrics,choice_3-value,choice_3-invalid_metrics
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,การเลื่อนการชำระเงินจากลูกหนี้บางรายอาจส่งผลกร...,Have impact,หากลูกหนี้บางรายขอเลื่อนการชำระเงิน อาจส่งผลกร...,Have impact,การที่ลูกหนี้บางรายขอเลื่อนการชำระเงินอาจมีผลก...,Have impact
1,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,มีความเสี่ยงจากการรับหรือเปิดอีเมล์จากภายนอก ซ...,Have single risk event,การรับและเปิดอีเมล์จากภายนอกมีความเสี่ยงที่อาจ...,Have single risk event,ความเสี่ยงในการรับและเปิดอีเมล์จากภายนอกอาจทำใ...,Have single risk event
2,อาจจะมีการเปิดไฟล์แนบมาในเมล์ที่มีไวรัสทำให้เก...,การเปิดไฟล์แนบในอีเมลอาจนำไปสู่การติดไวรัส ซึ่...,Have single risk event,หากมีการเปิดไฟล์แนบที่มีไวรัสในอีเมล จะทำให้ระ...,Have single risk event,การเปิดไฟล์แนบในอีเมลที่มีไวรัสอาจทำให้ระบบเกิ...,Have single risk event
3,เกิดการโจรกรรม ไฟล์และข้อมูลลูกค้าเปิดบริการเกมส์,การโจรกรรมข้อมูลลูกค้าในบริการเกมส์อาจส่งผลให้...,"Have single risk event, Have impact",การโจรกรรมไฟล์และข้อมูลลูกค้าในบริการเกมส์เป็น...,"Have single risk event, Have impact",การโจรกรรมข้อมูลลูกค้าในบริการเกมส์ถือเป็นเหตุ...,"Have single risk event, Have impact"
4,ระบบต้องได้รับการอัพเดต รวมไปถึงเทคโนโลยีใหม่ๆ...,การไม่อัพเดตระบบและเทคโนโลยีใหม่ๆ อาจทำให้เกิด...,"Have risk event, Have single risk event, Have ...",หากระบบไม่ได้รับการอัพเดตอย่างสม่ำเสมอ จะมีควา...,"Have risk event, Have single risk event, Have ...",การไม่ปรับปรุงระบบและการเรียนรู้ด้านไอทีอาจทำใ...,"Have risk event, Have single risk event, Have ..."
...,...,...,...,...,...,...,...
210,Concentration ของ source of revenue กับลูกค้าใ...,The risk associated with revenue concentration...,"Have risk event, Have single risk event, Have ...",There is a critical risk stemming from the con...,"Have risk event, Have single risk event, Have ...",The concentration of revenue from CPG group cu...,"Have risk event, Have single risk event, Have ..."
211,ลูกค้า (end customer) ซื้อสินค้าน้อยลง และกลุ่...,ลูกค้า (end customer) มีแนวโน้มในการซื้อสินค้า...,Have single risk event,การลดลงของการซื้อสินค้าจากลูกค้า (end customer...,Have single risk event,ลูกค้า (end customer) ประสบปัญหาการซื้อสินค้าล...,Have single risk event
212,COVID 19,The COVID-19 pandemic represents a significant...,"Have risk event, Have single risk event, Have ...",COVID-19 is a critical risk event that has led...,"Have risk event, Have single risk event, Have ...",The emergence of COVID-19 constitutes a major ...,"Have risk event, Have single risk event, Have ..."
213,"Analytics, Media, and CRM services are commonl...",The risk associated with the provision of Anal...,"Have risk event, Have single risk event, Have ...","In the competitive landscape of Analytics, Med...","Have risk event, Have single risk event, Have ...","The delivery of Analytics, Media, and CRM serv...","Have risk event, Have single risk event, Have ..."


In [84]:
all_feedback_df
# change original text to original_text
all_feedback_df.rename(columns={"text": "original_text"}, inplace=True)
all_feedback_df

,original text,raw_data,data_as_text
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,"{""summary_paragraph"": ""The user's sentence reg...",Summary: The user's sentence regarding debtors...
1,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
2,อาจจะมีการเปิดไฟล์แนบมาในเมล์ที่มีไวรัสทำให้เก...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
3,เกิดการโจรกรรม ไฟล์และข้อมูลลูกค้าเปิดบริการเกมส์,"{""summary_paragraph"": ""The user's sentence des...",Summary: The user's sentence describes a risk ...
4,ระบบต้องได้รับการอัพเดต รวมไปถึงเทคโนโลยีใหม่ๆ...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not meet the ...
...,...,...,...
210,Concentration ของ source of revenue กับลูกค้าใ...,"{""summary_paragraph"": ""The user's sentence reg...",Summary: The user's sentence regarding the con...
211,ลูกค้า (end customer) ซื้อสินค้าน้อยลง และกลุ่...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
212,COVID 19,"{""summary_paragraph"": ""The user sentence 'COVI...",Summary: The user sentence 'COVID 19' did not ...
213,"Analytics, Media, and CRM services are commonl...","{""summary_paragraph"": ""The user's sentence fai...",Summary: The user's sentence fails to meet the...


In [85]:
import pandas as pd

data = new_all_result

df = pd.DataFrame(data)

all_new_columns = {}

for index, row in df.iterrows():
    original_text = row["text"]
    new_columns = all_new_columns.get(original_text, {})
    metric_name = row["metric"]
    new_columns[f"metric_name-{metric_name}"] = new_columns.get(
        f"metric_name-{metric_name}", metric_name
    )
    new_columns[f"metric_detail-{metric_name}"] = new_columns.get(
        f"metric_detail-{metric_name}", row["metric_detail"]
    )
    new_columns[f"is_valid-{metric_name}"] = new_columns.get(
        f"is_valid-{metric_name}", row["is_valid"]
    )
    new_columns[f"reason-{metric_name}"] = new_columns.get(
        f"reason-{metric_name}", row["reason"]
    )
    new_columns[f"additional_information-{metric_name}"] = new_columns.get(
        f"additional_information-{metric_name}", row["additional_information"]
    )
    new_columns[f"human_is_valid-{metric_name}"] = new_columns.get(
        f"human_is_valid-{metric_name}", None
    )
    new_columns[f"human_reason-{metric_name}"] = new_columns.get(
        f"human_reason-{metric_name}", None
    )
    all_new_columns[original_text] = new_columns

In [86]:
new_all_result_df = pd.DataFrame(all_new_columns).T
new_all_result_df = new_all_result_df.reset_index()
new_all_result_df = new_all_result_df.rename(columns={"index": "original_text"})

new_all_result_df

,original_text,metric_name-Have risk event,metric_detail-Have risk event,is_valid-Have risk event,reason-Have risk event,additional_information-Have risk event,human_is_valid-Have risk event,human_reason-Have risk event,metric_name-Have single risk event,metric_detail-Have single risk event,...,additional_information-Have single risk event,human_is_valid-Have single risk event,human_reason-Have single risk event,metric_name-Have impact,metric_detail-Have impact,is_valid-Have impact,reason-Have impact,additional_information-Have impact,human_is_valid-Have impact,human_reason-Have impact
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes a specific risk sce...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,None,Have impact,Text describes potential consequences or outco...,False,The text states that some debtors may request ...,None,None,None
1,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes a specific risk sce...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,None,Have impact,Text describes potential consequences or outco...,True,The text describes a potential consequence of ...,None,None,None
2,การถูกโจมตีจากกระบวนการของคนนอก เช่น โจมตี server,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes a specific risk sce...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,None,Have impact,Text describes potential consequences or outco...,True,"The text describes a potential risk event, spe...",None,None,None
3,อาจจะมีการเปิดไฟล์แนบมาในเมล์ที่มีไวรัสทำให้เก...,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes a specific risk sce...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,None,Have impact,Text describes potential consequences or outco...,True,The text describes a potential consequence of ...,None,None,None
4,ความเสี่ยงต่อการโจมตีอุปกรณ์ไอทีภายในองค์กร ที...,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes a specific risk sce...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,None,Have impact,Text describes potential consequences or outco...,True,The text describes a specific risk related to ...,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,Concentration ของ source of revenue กับลูกค้าใ...,Have risk event,Text clearly describes what could go wrong or ...,False,The text mentions 'Concentration ของ source of...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,None,Have impact,Text describes potential consequences or outco...,False,The text mentions 'Concentration ของ source of...,None,None,None
269,ลูกค้า (end customer) ซื้อสินค้าน้อยลง และกลุ่...,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes a specific risk sce...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,None,Have impact,Text describes potential consequences or outco...,True,The text describes potential consequences of a...,None,None,None
270,COVID 19,Have risk event,Text clearly describes what could go wrong or ...,False,The text 'COVID 19' does not clearly describe ...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,None,Have impact,Text describes potential consequences or outco...,False,The text 'COVID 19' does not describe potentia...,None,None,None
271,"Analytics, Media, and CRM services are common

In [87]:
all_feedback_df
# change original text to original_text
all_feedback_df.rename(columns={"original text": "original_text"}, inplace=True)
all_feedback_df

,original_text,raw_data,data_as_text
0,ลูกหนี้บางรายจะขอเลื่อนการชำระเงิน,"{""summary_paragraph"": ""The user's sentence reg...",Summary: The user's sentence regarding debtors...
1,มีความเสี่ยงเรื่องการรับอีเมล์จากภายนอก หรือ เ...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
2,อาจจะมีการเปิดไฟล์แนบมาในเมล์ที่มีไวรัสทำให้เก...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
3,เกิดการโจรกรรม ไฟล์และข้อมูลลูกค้าเปิดบริการเกมส์,"{""summary_paragraph"": ""The user's sentence des...",Summary: The user's sentence describes a risk ...
4,ระบบต้องได้รับการอัพเดต รวมไปถึงเทคโนโลยีใหม่ๆ...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not meet the ...
...,...,...,...
210,Concentration ของ source of revenue กับลูกค้าใ...,"{""summary_paragraph"": ""The user's sentence reg...",Summary: The user's sentence regarding the con...
211,ลูกค้า (end customer) ซื้อสินค้าน้อยลง และกลุ่...,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
212,COVID 19,"{""summary_paragraph"": ""The user sentence 'COVI...",Summary: The user sentence 'COVID 19' did not ...
213,"Analytics, Media, and CRM services are commonl...","{""summary_paragraph"": ""The user's sentence fai...",Summary: The user's sentence fails to meet the...


In [88]:
# TODO : merge all df with original_text then add risk_name to it

In [89]:
dict_of_dataframes = {
    "label_metric": new_all_result_df,
    "improved_example_sentences": new_all_example_text_df,
    "feedback_messages": all_feedback_df,
}

In [90]:
from functools import reduce

dfs = list(dict_of_dataframes.values())


# Define a function to merge two DataFrames on 'original_text' using an outer join
def merge_df(left, right):
    return pd.merge(left, right, on="original_text", how="outer")


# Use reduce to apply the merge_df function cumulatively to the list of DataFrames
merged_df = reduce(merge_df, dfs)

merged_df

,original_text,metric_name-Have risk event,metric_detail-Have risk event,is_valid-Have risk event,reason-Have risk event,additional_information-Have risk event,human_is_valid-Have risk event,human_reason-Have risk event,metric_name-Have single risk event,metric_detail-Have single risk event,...,human_is_valid-Have impact,human_reason-Have impact,choice_1-value,choice_1-invalid_metrics,choice_2-value,choice_2-invalid_metrics,choice_3-value,choice_3-invalid_metrics,raw_data,data_as_text
0,(เป้าหมายของ BU) การลงทุนในธุรกิจไม่เป็นไปตามย...,Have risk event,Text clearly describes what could go wrong or ...,False,The text describes a potential risk scenario r...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,The business unit's investment strategy is at ...,"Have risk event, Have single risk event, Have ...",There is a critical risk associated with the b...,"Have risk event, Have single risk event, Have ...",The current investment approach of the busines...,"Have risk event, Have single risk event, Have ...","{""summary_paragraph"": ""The user's sentence fai...",Summary: The user's sentence fails to meet sev...
1,-การวางแผนและคาดการณ์ วิเคราะห์ข้อมูลตลาดอย่าง...,Have risk event,Text clearly describes what could go wrong or ...,False,The text does not clearly describe specific ri...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,The risk of inaccurate market forecasting may ...,"Have risk event, Have single risk event, Have ...",A significant risk event arises from the poten...,"Have risk event, Have single risk event, Have ...",The risk of supply chain disruptions could lea...,"Have risk event, Have single risk event, Have ...","{""summary_paragraph"": ""The user's sentence lac...",Summary: The user's sentence lacks clarity and...
2,1. อุทกภัยฯ (น้ำท่วม) น้ำท่วมโรงงาน 2. วาตภัย ...,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes specific risk scena...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,1. อุทกภัย (น้ำท่วม) ที่ส่งผลกระทบต่อโรงงาน โด...,Have single risk event,1. อุทกภัย (น้ำท่วม) ที่เกิดขึ้นในโรงงาน ทำให้...,Have single risk event,1. อุทกภัย (น้ำท่วม) ที่มีผลกระทบต่อการดำเนินง...,Have single risk event,"{""summary_paragraph"": ""The user's sentences di...",Summary: The user's sentences did not pass the...
3,1.การทุจริตในกระบวนการขาย การเบิกจ่าย ของส่งเส...,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes specific risk scena...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,1. การทุจริตในกระบวนการขายและการเบิกจ่ายของส่ง...,"Have single risk event, Have impact",1. การทุจริตในกระบวนการขายและการเบิกจ่ายของส่ง...,"Have single risk event, Have impact",1. การทุจริตในกระบวนการขายและการเบิกจ่ายของส่ง...,"Have single risk event, Have impact","{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not meet the ...
4,1.ผลิตและส่งมอบสินค้าไม่ได้ เนื่องจากโรงงานไม่...,Have risk event,Text clearly describes what could go wrong or ...,True,The text clearly describes specific risk scena...,None,None,None,Have single risk event,"Text describes a single, specific risk event r...",...,None,None,The inability to produce and deliver products ...,Have single risk event,Production and delivery of goods are compromis...,Have single risk event,The factory's non-compliance with established ...,Have single risk event,"{""summary_paragraph"": ""The user's sentence did...",Summary: The user's sentence did not pass the ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,ไม่ปฏิบัติตามหลักจริยธรรมทางธุรกิจ กับผู้มีส่ว...,Have risk event,Text clearly describes what could go wrong or ...,False,The text translates to 'Not adhering to busine...,None,None,None,Have 

In [91]:
# rename column raw_data -> feedback_raw_data
merged_df = merged_df.rename(
    columns={
        "original_text": "risk_description",
        "raw_data": "feedback_raw_data",
        "data_as_text": "feedback_data_as_text",
    }
)
# merged_df columns risk_name from risk_name_dict with key risk_description
merged_df["risk_name"] = merged_df["risk_description"].map(risk_name_dict)

In [92]:
merged_df.columns

Index(['risk_description', 'metric_name-Have risk event',
       'metric_detail-Have risk event', 'is_valid-Have risk event',
       'reason-Have risk event', 'additional_information-Have risk event',
       'human_is_valid-Have risk event', 'human_reason-Have risk event',
       'metric_name-Have single risk event',
       'metric_detail-Have single risk event',
       'is_valid-Have single risk event', 'reason-Have single risk event',
       'additional_information-Have single risk event',
       'human_is_valid-Have single risk event',
       'human_reason-Have single risk event', 'metric_name-Have impact',
       'metric_detail-Have impact', 'is_valid-Have impact',
       'reason-Have impact', 'additional_information-Have impact',
       'human_is_valid-Have impact', 'human_reason-Have impact',
       'choice_1-value', 'choice_1-invalid_metrics', 'choice_2-value',
       'choice_2-invalid_metrics', 'choice_3-value',
       'choice_3-invalid_metrics', 'feedback_raw_data',
       'fe

In [93]:
# reorder columns
columns_order = [
    "risk_name",
    "risk_description",
    "is_valid-Have risk event",
    "reason-Have risk event",
    "human_is_valid-Have risk event",
    "human_reason-Have risk event",
    "is_valid-Have single risk event",
    "reason-Have single risk event",
    "human_is_valid-Have single risk event",
    "human_reason-Have single risk event",
    "is_valid-Have impact",
    "reason-Have impact",
    "human_is_valid-Have impact",
    "human_reason-Have impact",
    "choice_1-value",
    "choice_2-value",
    "choice_3-value",
    "feedback_raw_data",
    "feedback_data_as_text",
    "choice_1-invalid_metrics",
    "choice_2-invalid_metrics",
    "choice_3-invalid_metrics",
    "metric_name-Have risk event",
    "metric_detail-Have risk event",
    "additional_information-Have risk event",
    "metric_name-Have single risk event",
    "metric_detail-Have single risk event",
    "additional_information-Have single risk event",
    "metric_name-Have impact",
    "metric_detail-Have impact",
    "additional_information-Have impact",
]
# reorder columns
merged_df = merged_df[columns_order]

In [94]:
merged_df.shape

(273, 31)

In [95]:
# merged_df
#

In [96]:
def save_dataframe_to_excel(
    df, part_ind, output_dir, excel_filename=None, sheet_name="Sheet1"
):
    """
    Saves a Pandas DataFrame to a single sheet in an Excel file.

    Args:
        df (pd.DataFrame): The Pandas DataFrame to save.
        excel_filename (str, optional): The name of the Excel file to create.
                                         Defaults to 'output.xlsx'.
        sheet_name (str, optional): The name of the sheet in the Excel file.
                                     Defaults to 'Sheet1'.

    Returns:
        None
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    if excel_filename is None:
        time_stampe = time.strftime("%Y%m%d-%H%M%S")
        excel_filename = f"{output_dir}/output_{time_stampe}_part{part_ind:02d}.xlsx"
    if not isinstance(df, pd.DataFrame):
        raise TypeError("Input 'df' must be a Pandas DataFrame.")
    if not isinstance(sheet_name, str):
        raise TypeError("Input 'sheet_name' must be a string.")

    try:
        df.to_excel(excel_filename, sheet_name=sheet_name, index=False)
        print(
            f"Successfully saved the DataFrame to '{excel_filename}' in sheet '{sheet_name}'."
        )
    except Exception as e:
        print(f"An error occurred while saving to Excel: {e}")

In [97]:
output_dir = "output_data"

In [98]:
save_dataframe_to_excel(merged_df, part_ind, output_dir)

Successfully saved the DataFrame to 'output_data/output_20250423-143516_part00.xlsx' in sheet 'Sheet1'.


In [99]:
import time
import json

# save fail_list to json file
time_stampe = time.strftime("%Y%m%d-%H%M%S")
with open(f"{output_dir}/fail_list_{time_stampe}_part{part_ind:02d}.json", "w") as f:
    json.dump(fail_list, f)

In [100]:
merged_df.columns.tolist()

['risk_name',
 'risk_description',
 'is_valid-Have risk event',
 'reason-Have risk event',
 'human_is_valid-Have risk event',
 'human_reason-Have risk event',
 'is_valid-Have single risk event',
 'reason-Have single risk event',
 'human_is_valid-Have single risk event',
 'human_reason-Have single risk event',
 'is_valid-Have impact',
 'reason-Have impact',
 'human_is_valid-Have impact',
 'human_reason-Have impact',
 'choice_1-value',
 'choice_2-value',
 'choice_3-value',
 'feedback_raw_data',
 'feedback_data_as_text',
 'choice_1-invalid_metrics',
 'choice_2-invalid_metrics',
 'choice_3-invalid_metrics',
 'metric_name-Have risk event',
 'metric_detail-Have risk event',
 'additional_information-Have risk event',
 'metric_name-Have single risk event',
 'metric_detail-Have single risk event',
 'additional_information-Have single risk event',
 'metric_name-Have impact',
 'metric_detail-Have impact',
 'additional_information-Have impact']